In [ ]:
sc

In [ ]:
import warnings
warnings.filterwarnings("ignore")

STATION = 235
FREQ = '4w'
PERIOD = 13

# Functions and helpers

In [ ]:
from dateutil.parser import parse 
from pyspark.sql.types import *
from pyspark.sql.functions import *
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller, kpss
from arch.unitroot import PhillipsPerron
import seaborn as sns
import numpy as np
import pandas as pd

plt.rcParams.update({'figure.figsize': (10, 7), 'figure.dpi': 120})

def get_specific_station_data(station, freq, filter_year = 1951):
    hadoopUrl = 'hdfs://hadoop-vm.internal.cloudapp.net:9000'
    data_files = f'{hadoopUrl}/precipitation/data/{station}/*.parquet'

    # Obtain dataset
    df = spark.read.parquet(data_files) \
            .withColumn("precipitation", col("precipitation").cast("float")) \
            .select("date","precipitation") \
            .toPandas()
    
    # Set the date column as the index and ensure it's a DatetimeIndex
    df.set_index('date', inplace=True)
    df.index = pd.to_datetime(df.index)
    
    # Filter years
    df = df[df.index.year > filter_year] # Only 1951 onwards
    
    # Set the frequency data set
    df = df.resample(freq).mean()
    
    # drop null
    df.reset_index(inplace=True)
    df = df.dropna(axis=0)
    
    # set index
    df.set_index('date', inplace=True)
    df = df.sort_index()
    return df


def test_stationarity(time_series):
    # Perform ADF test
    adf_result = adfuller(time_series, autolag='AIC')
    print("ADF Test:")
    print("=========")
    print(f"Test statistic: {adf_result[0]}")
    print(f"null_hypothesis: the time series is non-stationary")
    p_value = adf_result[1]
    print(f"P-value: {p_value}")
    print("Critical values:")
    for key, value in adf_result[4].items():
        print(f"{key}: {value}")
    
    result = "STATIONARY" if p_value < 0.05 else "NON-STATIONARY"
    print(result)
    

    print("\nKPSS Test:")
    print("============")
    kpss_result = kpss(time_series, regression='ct')  # 'ct' for constant and trend
    print(f"null_hypothesis: the time series is trend stationary")
    print(f"Test statistic: {kpss_result[0]}")
    print(f"P-value: {kpss_result[1]}")
    p_value = kpss_result[1]
    print("Critical values:")
    for key, value in kpss_result[3].items():
        print(f"{key}: {value}")
        
    result = "NON-STATIONARY" if p_value < 0.05 else "STATIONARY"
    print(result)


def remove_outlier(df):
    # Calculate the IQR
    Q1 = df['precipitation'].quantile(0.25)
    Q3 = df['precipitation'].quantile(0.75)
    IQR = Q3 - Q1

    # Define the bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out the outliers
    output = df[(df['precipitation'] >= lower_bound) & (df['precipitation'] <= upper_bound)]
    print(f"Records removed: {df.shape[0] - output.shape[0]}")
    return output


# SARIMA

## Stationary test

Refer to Data preparation

## Model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error

df_seasson = get_specific_station_data(STATION,FREQ,1800)

scaler = MinMaxScaler() # So it can be compared with LSTM
df_seasson = scaler.fit_transform(df_seasson)

train_data, test_data = train_test_split(df_seasson, test_size=0.2,shuffle=False)


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
plot_acf(train_data, lags=20, ax=ax1)
plot_pacf(train_data, lags=20, ax=ax2)
plt.show()


In [ ]:
p = 1
d = 0
q = 1
s = PERIOD  # Seasonal period (e.g., 12 months)
P = 1
D = 1
Q = 1

model = SARIMAX(train_data, order=(p, d, q), seasonal_order=(P, D, Q, s))
results = model.fit()


In [ ]:
predictions = results.get_forecast(steps=len(test_data))
mean_predictions = predictions.predicted_mean

# Calculate evaluation metrics
test_rmse = np.sqrt(mean_squared_error(test_data, mean_predictions))
test_mae = mean_absolute_error(test_data, mean_predictions)
#test_mape = np.mean(np.abs((test_data - mean_predictions) / test_data)) * 100
#test_mase = test_mae / (np.mean(np.abs(test_data[1:] - test_data[:-1])))

print(f"Test RMSE: {test_rmse:.2f}")
print(f"Test MAE: {test_mae:.2f}")
#print(f"Test MAPE: {test_mape:.2f}%")
#print(f"Test MASE: {test_mase:.2f}")



In [ ]:
plt.figure(figsize=(12, 6))
#plt.plot(train_data, label='Training Data')
plt.plot(test_data, label='Actual values')
plt.plot(mean_predictions, label='Predictions', linestyle='dashed')
plt.xlabel('Time')
plt.ylabel('Precipitation')
plt.legend()
plt.show()


## AUTO ARIMA

In [ ]:
#%pip install pmdarima

In [ ]:
import pmdarima as pm

scaler = MinMaxScaler()

df_seasson = get_specific_station_data(STATION,FREQ,1800)
df_seasson = scaler.fit_transform(df_seasson)
train_data, test_data = train_test_split(df_seasson, test_size=0.2,shuffle=False)

auto_model = pm.auto_arima(train_data,
                      seasonal=True,
                      m=PERIOD,  # Seasonal frequency
                      start_p=0, start_q=0, max_p=3, max_q=3,  # Non-seasonal parameters
                      start_P=0, start_Q=0, max_P=3, max_Q=3,  # Seasonal parameters
                      d=0, D=3,  # Orders of differencing
                      trace=False,  # Print search information
                      error_action='ignore',
                      suppress_warnings=True,
                      stepwise=True)  # Stepwise search for efficiency

In [ ]:
print(f"Best SARIMA Model: {auto_model.order}, {auto_model.seasonal_order}")

In [ ]:
model = SARIMAX(train_data, order=auto_model.order, seasonal_order=auto_model.seasonal_order)
results = model.fit()

In [ ]:
predictions = results.get_forecast(steps=len(test_data))
mean_predictions = predictions.predicted_mean

rmse = np.sqrt(mean_squared_error(test_data, mean_predictions))
print(f"RMSE: {rmse}")


In [ ]:
plt.figure(figsize=(12, 6))
#plt.plot(train_data, label='Training Data')
plt.plot(test_data, label='Actual values')
plt.plot(mean_predictions, label='Predictions', linestyle='dashed')
plt.xlabel('Time')
plt.ylabel('Precipitation')
plt.legend()
plt.show()